In [6]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [278]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (0, 0, 1)
r2 = (2, 1, 1)
r3 = (3, 2, 1)
r4 = (4, 4, 1)

floaters = np.hstack((r1, r2, r3, r4))

m = 4             # total atoms in the chain

In [279]:
# define parameters
beta = 1

In [280]:
# define time interval
t = np.linspace(0,6,10)

In [281]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((m,3))
    total_bending_force = np.zeros(m)
    
    # loop for computing bending force
    for i in range(1,m-1):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            print("try")
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            current_bending_force[i][0] += Eb_common*(B*Axn - A*Bxn)
            current_bending_force[i][1] += Eb_common*(B*Ayn - A*Byn)
            current_bending_force[i][2] += Eb_common*(B*Azn - A*Bzn)
            
            current_bending_force[i-1][0] += Eb_common*(B*Axn0 - A*Bxn0)
            current_bending_force[i-1][1] += Eb_common*(B*Ayn0 - A*Byn0)
            current_bending_force[i-1][2] += Eb_common*(B*Azn0 - A*Bzn0)
            
            current_bending_force[i+1][0] = Eb_common*(B*Axn1 - A*Bxn1)
            current_bending_force[i+1][1] = Eb_common*(B*Ayn1 - A*Byn1)
            current_bending_force[i+1][2] = Eb_common*(B*Azn1 - A*Bzn1)
            
        except Exception as error:
            print("except")
            print("Caught exception: ", error)
            continue
            

    total_bending_force = np.hstack(current_bending_force)
    print(total_bending_force)
    return total_bending_force

In [282]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[-0.06662043  0.13324087  0.          0.06662043 -0.13324087  0.
  0.13324087 -0.06662043  0.         -0.13324087  0.06662043  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[-0.06662054  0.13324145  0.          0.06662054 -0.13324145  0.
  0.13324145 -0.06662054  0.         -0.13324145  0.06662054  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[-0.06662054  0.13324145  0.          0.06662054 -0.13324145  0.
  0.13324145 -0.06662054  0.         -0.13324145  0.06662054  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[-0.06662064  0.13324203  0.          0.06662064 -0.13324203  0.
  0.13324203 -0.06662064  0.         -0.13324203  0.06662064  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[-0.06662064  0.13324203  0.          0.06662064 -0.13324203  0.
  0.13324203 -

[ 0.11895936  0.60176419  0.         -0.11895936 -0.60176419  0.
  0.60176419  0.11895936  0.         -0.60176419 -0.11895936  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 0.14293508  0.62843242  0.         -0.14293508 -0.62843242  0.
  0.62843242  0.14293508  0.         -0.62843242 -0.14293508  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 0.14293508  0.62843242  0.         -0.14293508 -0.62843242  0.
  0.62843242  0.14293508  0.         -0.62843242 -0.14293508  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 0.17030574  0.65710521  0.         -0.17030574 -0.65710521  0.
  0.65710521  0.17030574  0.         -0.65710521 -0.17030574  0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 0.17030576  0.65710525  0.         -0.17030576 -0.65710525  0.
  0.65710525  0.17030576  0.         -0.65710525 -0.17030576  0.        ]
[[ 0.  0.  0.

try
2
try
[ 6.20183326  1.58744358  0.         -6.20183326 -1.58744357  0.
  1.58744359  6.20183327 -0.         -1.58744358 -6.20183328 -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 6.68714698  1.54452375  0.         -6.68714699 -1.54452374  0.
  1.54452376  6.687147   -0.         -1.54452375 -6.68714701 -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 6.68715226  1.54452352  0.         -6.68715227 -1.54452351  0.
  1.54452353  6.68715227 -0.         -1.54452352 -6.68715228 -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 7.25992019  1.48379374  0.         -7.2599202  -1.48379373  0.
  1.48379375  7.25992021 -0.         -1.48379374 -7.25992022 -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 7.2599274   1.48379327  0.         -7.25992741 -1.48379325  0.
  1.48379328  7.25992742 -0.         -1.48379326 -7.25992743 -0.        ]
[[ 

try
2
try
[ 236.41270277 -124.97878136    0.         -236.41270748  124.97878606
    0.         -124.97878553  236.4127115    -0.          124.97879024
 -236.41271621   -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 236.41344242 -124.97927028    0.         -236.41344713  124.97927499
    0.         -124.97927446  236.41345115   -0.          124.97927916
 -236.41345586   -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 231.91108318 -121.99724784    0.         -231.91108772  121.99725238
    0.         -121.99725184  231.91109161   -0.          121.99725638
 -231.91109615   -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 231.91139912 -121.99745605    0.         -231.91140366  121.99746059
    0.         -121.99746005  231.91140754   -0.          121.99746459
 -231.91141208   -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 250.5222

try
2
try
[ 48295.97230673 -43880.64201518      0.         -48296.08700159
  43880.75671004      0.         -43880.81397631  48296.16171961     -0.
  43880.92867117 -48296.27641448     -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 47143.16603396 -42799.44017728      0.         -47143.27562491
  42799.54976822      0.         -42799.60440665  47143.34707662     -0.
  42799.7139976  -47143.45666756     -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 47143.41189557 -42799.67070366      0.         -47143.52148846
  42799.78029655      0.         -42799.83493598  47143.59294143     -0.
  42799.94452886 -47143.70253432     -0.        ]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[ 51900.15490944 -47264.48293379      0.         -51900.28625551
  47264.61427987      0.         -47264.68013842  51900.37161446     -0.
  47264.81148449 -51900.50296053     -0.        ]
[[ 0.  0.  0.]
 [ 0

...

try
2
try
[  9.46196086e+10  -9.45424160e+10   0.00000000e+00   7.48487258e+22
  -7.48440488e+22   0.00000000e+00   1.18346244e+23  -1.18338818e+23
  -0.00000000e+00  -1.93194971e+23   1.93182865e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46259643e+10  -9.45487682e+10   0.00000000e+00   7.48487258e+22
  -7.48440488e+22   0.00000000e+00   1.18346244e+23  -1.18338818e+23
  -0.00000000e+00  -1.93194971e+23   1.93182865e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46150272e+10  -9.45378372e+10   0.00000000e+00   7.48487258e+22
  -7.48440488e+22   0.00000000e+00   1.18346244e+23  -1.18338818e+23
  -0.00000000e+00  -1.93194971e+23   1.93182865e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46196086e+10  -9.45424160e+10  -2.36564102e+06   7.48487258e+22
  -7.48440488e+22   1.44913058e+06   1.18346244e+23  -1.18338818e+23
   9.1651

2
try
[  9.46278127e+10  -9.45506151e+10   1.44929814e+06   2.45864163e+22
  -2.45848779e+22  -1.92749606e+21   3.88745415e+22  -3.88721040e+22
  -3.04763897e+21  -6.34609580e+22   6.34569823e+22   4.97513503e+21]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46219108e+10  -9.45447181e+10   0.00000000e+00   3.53737364e+22
  -3.53715240e+22   0.00000000e+00   5.59307932e+22  -5.59272862e+22
  -0.00000000e+00  -9.13045294e+22   9.12988113e+22  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46330266e+10  -9.45558248e+10   0.00000000e+00   1.22955300e+22
  -1.22947604e+22   0.00000000e+00   1.94409418e+22  -1.94397217e+22
  -0.00000000e+00  -3.17364716e+22   3.17344823e+22  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46278127e+10  -9.45506151e+10   9.16616409e+05   2.45864163e+22
  -2.45848779e+22  -3.04763897e+21   3.88745415e+22  -3.88721040e+22
  -4.81874049

2
try
except
Caught exception:  Divide by zero
[  9.46111727e+10  -9.45339851e+10   0.00000000e+00  -5.79701660e+10
   5.78952705e+10   0.00000000e+00  -3.66410067e+10   3.66387145e+10
  -0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
except
Caught exception:  Divide by zero
[  9.46157533e+10  -9.45385631e+10  -2.36551250e+06  -5.79729726e+10
   5.78980747e+10   1.44905185e+06  -3.66427807e+10   3.66404884e+10
   9.16460645e+05   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46152974e+10  -9.45381073e+10   0.00000000e+00  -1.35974346e+23
   1.35965825e+23   0.00000000e+00  -2.14994280e+23   2.14980895e+23
  -0.00000000e+00   3.50968626e+23  -3.50946712e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46163848e+10  -9.45391943e+10   0.00000000e+00  -1.12341806e+23


 [ 0.  0.  0.]]
1
try
2
try
except
Caught exception:  Divide by zero
[  9.46156823e+10  -9.45384921e+10   0.00000000e+00  -5.79729291e+10
   5.78980312e+10   0.00000000e+00  -3.66427532e+10   3.66404609e+10
  -0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46149095e+10  -9.45377198e+10   0.00000000e+00  -2.19222615e+23
   2.19208924e+23   0.00000000e+00  -3.46621455e+23   3.46599711e+23
  -0.00000000e+00   5.65844057e+23  -5.65808631e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46145838e+10  -9.45373942e+10   0.00000000e+00  -2.55184020e+23
   2.55168073e+23   0.00000000e+00  -4.03481418e+23   4.03456126e+23
  -0.00000000e+00   6.58665434e+23  -6.58624199e+23  -0.00000000e+00]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
try
2
try
[  9.46209397e+10  -9.45437466e+10   0.00000000e+00  -2.55184020e+23
   2.55168073e+23  

/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [283]:
print(gFlow)

[[  0.00000000e+000   0.00000000e+000   1.00000000e+000   2.00000000e+000
    1.00000000e+000   1.00000000e+000   3.00000000e+000   2.00000000e+000
    1.00000000e+000   4.00000000e+000   4.00000000e+000   1.00000000e+000]
 [ -4.63830122e-002   1.06194730e-001   1.00000000e+000   2.04638301e+000
    8.93805270e-001   1.00000000e+000   3.10619473e+000   1.95361699e+000
    1.00000000e+000   3.89380527e+000   4.04638301e+000   1.00000000e+000]
 [ -9.18056689e-002   2.59250383e-001   1.00000000e+000   2.09180567e+000
    7.40749617e-001   1.00000000e+000   3.25925038e+000   1.90819433e+000
    1.00000000e+000   3.74074962e+000   4.09180567e+000   1.00000000e+000]
 [ -1.17976970e-001   4.88709526e-001   1.00000000e+000   2.11797697e+000
    5.11290474e-001   1.00000000e+000   3.48870953e+000   1.88202303e+000
    1.00000000e+000   3.51129047e+000   4.11797697e+000   1.00000000e+000]
 [ -5.84811176e-002   8.60024555e-001   1.00000000e+000   2.05848112e+000
    1.39975445e-001   1.00000000e+

In [271]:
sx_n = gFlow[-1][3] - gFlow[-1][0]
sx_n1 = gFlow[-1][6] - gFlow[-1][3]

sy_n = gFlow[-1][4] - gFlow[-1][1]
sy_n1 = gFlow[-1][7] - gFlow[-1][4]

sz_n = gFlow[-1][5] - gFlow[-1][2]
sz_n1 = gFlow[-1][8] - gFlow[-1][5]

A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

print(A, B, A-B, A+B, 2*beta*(A-B)/(A+B))

0.0 0.0 0.0 0.0 nan


/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
